In [1]:
import pandas as pd


This notebook represents experiments with the implementation of part of the code from the article "Snomed2Vec: RandomWalk and Poincaré Embeddings of a Clinical Knowledge Base for Healthcare Analytics"   

source: https://gitlab.com/agarwal.khushbu/Snomed2Vec/-/blob/master/src/embedding_learning/poincare.py 


In [8]:
df=pd.read_csv('C://Users/Nusha/Desktop/Passaustudying/fouth semester/master/master_thesis/snomed/CONCEPT_RELATIONSHIP.csv', sep='\t')

In [13]:
df[df.relationship_id=='Is a']

,concept_id_1,concept_id_2,relationship_id,valid_start_date,valid_end_date,invalid_reason
67,281,280,Is a,19700101,20991231,NaN
74,282,281,Is a,19700101,20991231,NaN
77,283,281,Is a,19700101,20991231,NaN
80,284,281,Is a,19700101,20991231,NaN
83,285,280,Is a,19700101,20991231,NaN
...,...,...,...,...,...,...
12034931,3661632,37311061,Is a,20210129,20991231,NaN
12034932,3655975,37311061,Is a,20210129,20991231,NaN
12034933,3661748,37311061,Is a,20210129,20991231,NaN
12034934,3655976,37311061,Is a,20210129,20991231,NaN


In [14]:
df_concept=pd.read_csv('C://Users/Nusha/Desktop/Passaustudying/fouth semester/master/master_thesis/snomed/CONCEPT.csv', sep='\t')

c:\Users\Nusha\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (6,9) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [16]:
df_concept[df_concept.concept_id==281]

,concept_id,concept_name,domain_id,vocabulary_id,concept_class_id,standard_concept,concept_code,valid_start_date,valid_end_date,invalid_reason
1241008,281,Medicare Managed Care,Payer,SOPT,Payer,S,11,19700101,20991231,NaN
